## Perform and pickle cross-validation

In [1]:
import numpy as np
import pickle
import warnings

import sparseRRR
from sklearn.exceptions import ConvergenceWarning

In [2]:
def preprocess(data):
    X = data['Firing rate']
    X = X - np.mean(X, axis=0)
    X = X / np.std(X, axis=0)

    Y = data['Locomotion']
    Y = Y - np.mean(Y, axis=0)
    Y = Y / np.std(Y, axis=0)
    
    return X,Y

## The main cross-validation setup

In [3]:
data = pickle.load(open('../data/purkinje.pickle', 'rb'))
X,Y = preprocess(data)
print('Shape of X:', X.shape, '\nShape of Y:', Y.shape)

Shape of X: (2691968, 105) 
Shape of Y: (2691968, 4)


It is a lot of timepoints, so let us play around with reduced data set sizes.

In [4]:
slice = np.linspace(0, 10000, 10000, dtype=int)

In [11]:
alphas = np.concatenate((np.arange(.04,1.01,.1), np.arange(2,5)))
l1_ratios = np.array([.25, .5, .75, 1])

with warnings.catch_warnings():
    warnings.simplefilter("ignore", category=(ConvergenceWarning, RuntimeWarning))
    cvresults = sparseRRR.elastic_rrr_cv(X[slice,:], Y[slice,:], rank=2, reps=1, folds=10, alphas=alphas, l1_ratios=l1_ratios)

l1_ratios = np.array([1])
ranks = np.arange(1, Y.shape[1]+1)

cvresults_rank = {}
with warnings.catch_warnings():
    warnings.simplefilter("ignore", category=(ConvergenceWarning, RuntimeWarning))
    for r in ranks:
        cvresults_rank[r] = sparseRRR.elastic_rrr_cv(X[slice,:], Y[slice,:], rank=r, reps=1, folds=10, alphas=alphas, l1_ratios=l1_ratios)
    
pickle.dump([cvresults, cvresults_rank], open('../pickles/cvresults-purkinje.pickle', 'wb'))

1.......... Time: 0.0h  2m 30s
1.......... Time: 0.0h  0m  8s
1.......... Time: 0.0h  0m 20s
1.......... Time: 0.0h  1m  7s
1.......... Time: 0.0h  0m 14s


#### Template code for nested CV

In [12]:
alphas = np.concatenate((np.arange(.04,1.01,.1), np.arange(2,4)))
l1_ratios = np.array([.25, .5, .75, 1])

with warnings.catch_warnings():
   warnings.simplefilter("ignore", category=(ConvergenceWarning, RuntimeWarning))
   sparseRRR.nested_cv(X[slice,:], Y[slice,:], alphas, l1_ratios, target_n_predictors=10)

1.......... Time: 0.0h  2m  0s
Optimal l1 ratio: 1.0, alpha to get 10 predictors: 0.2, test R2 = 0.16
1.......... Time: 0.0h  2m  2s
Optimal l1 ratio: 1.0, alpha to get 10 predictors: 0.2, test R2 = 0.59
1.......... Time: 0.0h  2m  1s
Optimal l1 ratio: 0.75, alpha to get 10 predictors: 0.4, test R2 = 0.58
1.......... Time: 0.0h  2m 17s
Optimal l1 ratio: 1.0, alpha to get 10 predictors: 0.2, test R2 = 0.67
1.......... Time: 0.0h  2m 11s
Optimal l1 ratio: 1.0, alpha to get 10 predictors: 0.3, test R2 = 0.71
1.......... Time: 0.0h  2m 20s
Optimal l1 ratio: 1.0, alpha to get 10 predictors: 0.2, test R2 = 0.42
1.......... Time: 0.0h  2m 15s
Optimal l1 ratio: 1.0, alpha to get 10 predictors: 0.2, test R2 = 0.61
1.......... Time: 0.0h  2m 12s
Optimal l1 ratio: 1.0, alpha to get 10 predictors: 0.2, test R2 = 0.68
1.......... Time: 0.0h  2m 23s
Optimal l1 ratio: 1.0, alpha to get 10 predictors: 0.2, test R2 = 0.14
1.......... Time: 0.0h  2m  3s
Optimal l1 ratio: 1.0, alpha to get 10 predictors: